<a href="https://colab.research.google.com/github/keelinarseneault/ML-Engineering/blob/main/Exploring_CNN_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import os

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
train_path = '/Users/karseneault/Desktop/train_data/'
test_path = '/Users/karseneault/Desktop/test_data_v2/'

In [4]:
train = pd.read_csv('drive/MyDrive/train.csv')
test = pd.read_csv('drive/MyDrive/test.csv')

In [5]:
train = train[['file_name', 'label']]
train.columns = ['id', 'label']

In [6]:
print(train.shape)

(79950, 2)


In [7]:
print(train.value_counts('label'))

label
0    39975
1    39975
Name: count, dtype: int64


# **Compare CNN Architectures**

## **ConvNeXT:**

In [9]:
# Use PyTorch

import pandas as pd
import os
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

In [10]:
path = 'drive/MyDrive/Images'

In [11]:
def image_exists(id):
    filepath = f"drive/MyDrive/Images/{id}"
    return os.path.isfile(filepath)

In [12]:
train = train[train["id"].apply(image_exists)]

In [12]:
train.shape

(11040, 2)

**Take a random sample of the training set in order to train on a smaller set of images, while maintaining the balanced ratio between the two classes:**

In [13]:
train_sample = train.groupby("label", group_keys=False).apply(lambda x:x.sample(frac=0.5))

<ipython-input-13-137aab85c240>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_sample = train.groupby("label", group_keys=False).apply(lambda x:x.sample(frac=0.5))


In [14]:
train_df, val_df = train_test_split(
    train_sample,
    test_size=0.05,
    random_state=42,
    stratify=train_sample['label']
)

In [26]:
# Print shapes of the splits
print(f'Train shape: {train_df.shape}')
print(f'Validation shape: {val_df.shape}')

Train shape: (5244, 2)
Validation shape: (276, 2)


In [27]:
class AIImageDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.dataframe.iloc[idx, 1]
        return image, label

In [28]:
train_transforms = transforms.Compose([
    transforms.Resize(232),  # Resize to match ConvNeXt preprocessing
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [29]:
train_dataset = AIImageDataset(train_df, root_dir=path, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

In [30]:
# Load pretrained ConvNeXt Base model
model = models.convnext_base(weights="DEFAULT")

# Freeze all layers initially
for param in model.features.parameters():
    param.requires_grad = False

# Unfreeze the last two stages
for param in model.features[-2:].parameters():
    param.requires_grad = True

# Replace the classifier head with a custom one
model.classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d((1, 1)),  # Global average pooling
    nn.Flatten(),                  # Flatten the tensor
    nn.BatchNorm1d(1024),          # Add BatchNorm here
    nn.Linear(1024, 512),          # First fully connected layer
    nn.ReLU(),                     # Activation function
    nn.Dropout(0.4),               # Dropout for regularization
    nn.Linear(512, 2)              # Output layer (binary classification)
)

optimizer = torch.optim.AdamW([
    {'params': model.features[-2:].parameters(), 'lr': 1e-5},  # Lower LR for backbone
    {'params': model.classifier.parameters(), 'lr': 1e-4}      # Higher LR for classifier
])

criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=5, gamma=0.7)

In [31]:
torch.cuda.is_available()


True

In [32]:
# Training Loop
epochs = 5

train_losses, train_accuracies, val_losses, val_accuracies, val_f1s = [], [], [], [], []

for epoch in range(epochs):
    # Training
    model.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0

    for data, label in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        preds = output.argmax(dim=1)
        acc = (preds == label).float().mean().item()
        epoch_accuracy += acc

    epoch_loss /= len(train_loader)
    epoch_accuracy /= len(train_loader)

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(
        f"Epoch [{epoch+1}/{epochs}] "
        f"Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_accuracy:.4f} | ")

  # Step the learning rate scheduler
    scheduler.step()

Training Epoch 1: 100%|██████████| 164/164 [45:18<00:00, 16.58s/it]


Epoch [1/5] Train Loss: 0.4323 | Train Acc: 0.8201 | 


Training Epoch 2: 100%|██████████| 164/164 [44:24<00:00, 16.25s/it]


Epoch [2/5] Train Loss: 0.2628 | Train Acc: 0.8989 | 


Training Epoch 3: 100%|██████████| 164/164 [41:49<00:00, 15.30s/it]


Epoch [3/5] Train Loss: 0.2163 | Train Acc: 0.9163 | 


Training Epoch 4: 100%|██████████| 164/164 [45:51<00:00, 16.78s/it]


Epoch [4/5] Train Loss: 0.1891 | Train Acc: 0.9246 | 


Training Epoch 5: 100%|██████████| 164/164 [46:10<00:00, 16.90s/it]

Epoch [5/5] Train Loss: 0.1868 | Train Acc: 0.9210 | 


## **InceptionV3:**

In [16]:
# Use TensorFlow

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [79]:
path = 'drive/MyDrive/Images/'

In [17]:
# Function to load and preprocess an image

def preprocess_image(image_path):
  img = load_img(image_path, target_size=(299, 299))  # Adjust target_size if needed
  img = img_to_array(img)
  img = img / 255.0  # Normalize pixel values
  img = np.expand_dims(img, axis=0)  # Add batch dimension
  return img

# Preprocess training data
train_images = []
train_labels = []
for index, row in train_df.iterrows():
    image_path = os.path.join(path, row['id'])
    train_images.append(preprocess_image(image_path))
    train_labels.append(row['label'])

train_images = np.array(train_images)
train_labels = np.array(train_labels)


In [18]:
# Prepare validation data

val_images = []
val_labels = []
for index, row in val_df.iterrows():
    image_path = os.path.join(path, row['id'])
    val_images.append(preprocess_image(image_path))
    val_labels.append(row['label'])

val_images = np.concatenate(val_images, axis=0)
val_labels = np.array(val_labels)

In [19]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

base_model.trainable = False

In [20]:
# Add layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)  # Prevent overfitting
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create final model
model = Model(inputs=base_model.input, outputs=output)

In [21]:
model.compile(optimizer = 'adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [ ]:
model_history = model.fit(train_images, train_labels, epochs=5, validation_data=(val_images, val_labels))

In [ ]:
# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(train_dataset)

print(f"Accuracy on train data: {train_accuracy:.2%} | Loss: {train_loss:.4f}")